## How to edit an already existing Neuron Model Configuration

This notebook explains how to load an already existing neuron model configuration from nexus, edit it, and store it on nexus

To run this notebook, you need to already have a NeuronModelConfiguration in Nexus. Therefore, I recommend to do the configuration through python before running this notebook:
```
python pipeline.py --step=configure_nexus --emodel=EMODEL_NAME --iteration_tag=ITERATION_TAG --ttype=TTYPE
python pipeline.py --step=configure_model_from_gene --emodel=EMODEL_NAME --iteration_tag=ITERATION_TAG --ttype=TTYPE
```

## Loading the NeuronModelConfiguration

In [1]:
species = "mouse"
brain_region = "SSCX"
data_access_point = "nexus"
nexus_project = "ncmv3"
nexus_organisation = "bbp"
nexus_endpoint = "prod"
forge_path = "./forge.yml"
forge_ontology_path = "./nsg.yml"

emodel = "L5_TPC:B_cAC"
ttype = "182_L4/5 IT CTX"
iteration_tag = "mettypesv12_1"

This next cell will ask for your Nexus token:

In [ ]:
from bluepyemodelnexus.nexus import NexusAccessPoint

access_point = NexusAccessPoint(
    emodel=emodel,
    species=species,
    brain_region=brain_region,
    project="ncmv3",
    organisation="bbp",
    endpoint="https://bbp.epfl.ch/nexus/v1",
    forge_path=forge_path,
    forge_ontology_path=forge_ontology_path,
    ttype=ttype,
    iteration_tag=iteration_tag
)

Here, we load the configurator, that is able to load, modify and save the configuration.

In [ ]:
from bluepyemodel.model.model_configurator import ModelConfigurator

configurator = ModelConfigurator(access_point)
configurator.load_configuration()

print(configurator.configuration)

## Adding Mechanisms, Distributions, Parameters or Morphology

You are now able to modify the neuron model configuration. Below is an example where a mechanism, a distribution and a parameter are added to the other mechanisms, distributions and parameters of the neuron model configuration, and where the morphology is changed to another one.

In [4]:
from bluepyemodel.model.morphology_configuration import MorphologyConfiguration

In [5]:
# add distribution
configurator.configuration.add_distribution(
    distribution_name="decay",
    function="math.exp({distance}*{constant})*{value}",
    parameters=["constant"],
)

In [6]:
# add two parameters
configurator.configuration.add_parameter(
    parameter_name="constant",
    locations="distribution_decay",
    value=[-0.1,0.0],
    mechanism=None,
    distribution_name=None,
    stochastic=None,
)
configurator.configuration.add_parameter(
    parameter_name="gNaTgbar_NaTg",
    locations="apical",
    value=[0.0, 0.1],
    mechanism="NaTg",
    distribution_name="decay",
    stochastic=None,
)

In [7]:
# add mechanism
configurator.configuration.add_mechanism(
    mechanism_name="NaTg",
    locations="apical",
    stochastic=None,
    version=None,
)

In [8]:
# change morphology
configurator.configuration.morphology = MorphologyConfiguration(name="ak171123_C_idA")

## Removing Parameters, Mechanisms, and Distributions

Here, we will see how to remove parameters, mechanisms and distributions. For the sake of simplicity, we will remove the ones that we just added.

In [15]:
# remove parameter
configurator.configuration.remove_parameter(parameter_name="constant")

When a mechanism is removed, it also remove all associated parameters. Here, 'gNaTgbar_NaTg' parameter will be removed as well

In [16]:
# remove mechanism
configurator.configuration.remove_mechanism(mechanism_name="NaTg")

There is no function to remove a distribution, but if you want to remove one nonetheless, you can use this code below.

In [17]:
# remove a distribution
distribution_name = "decay"
configurator.configuration.distributions = [d for d in configurator.configuration.distributions if d.name != distribution_name]

## Saving the NeuronModelConfiguration

If there is no optimised model in your access point, you can store the new neuron model configuration as it is.

However, if there is an optimised model in your access point, you might want to use a new access point with an unused iteration tag in ordrer to not have a conflict with the existing model.

In this example, since we have just created the access point without optimising any model yet, we will keep it for storing the new neuron model configuration. Note that the old NeuronModelConfiguration will be deprecated.

In [ ]:
# store new model config
configurator.save_configuration()

## Trying if the optimisation work:

Go to your folder containing the mechanisms (e.g. ./nexus_tmp/emodel=L5_TPC:B_cAC__iteration_tag=mettypesv12_1__ttype=182_L4/5_IT_CTX) and compile them. Then, you can try if the optimisation works using the pipeline:

```
extraction:
python pipeline.py --step=extract --emodel=EMODEL_NAME --iteration_tag=ITERATION_TAG --ttype=TTYPE
optimisation:
python pipeline.py --step=test_optimise --emodel=EMODEL_NAME --iteration_tag=ITERATION_TAG --ttype=TTYPE
```